In [1]:
# imports:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# what stuff do i need to crop 1 image and extract just the face?
# dataframe that you created!
# load the dataframe with pandas, read each image with openCV and save result to faces folder

dataframe = pd.read_csv('mask_df.csv')
dataframe

,image,class,bbox_x,bbox_y,bbox_w,bbox_h,kp1_x,kp1_y,kp2_x,kp2_y
0,images\mask\michal_mask_1.jpg,MASK,234,61,258,205,319,195,384,197
1,images\mask\michal_mask_10.jpg,MASK,234,51,247,191,290,163,350,167
2,images\mask\michal_mask_100.jpg,MASK,198,89,233,201,316,210,363,207
3,images\mask\michal_mask_11.jpg,MASK,242,57,261,209,323,199,379,201
4,images\mask\michal_mask_12.jpg,MASK,229,49,273,225,313,196,368,195
...,...,...,...,...,...,...,...,...,...,...
695,images\bad_mask\michal_bad_mask_95.jpg,BAD_MASK,182,76,246,212,206,160,262,173
696,images\bad_mask\michal_bad_mask_96.jpg,BAD_MASK,169,83,227,239,226,189,286,187
697,images\bad_mask\michal_bad_mask_97.jpg,BAD_MASK,228,84,208,179,258,174,323,178
698,images\bad_mask\michal_bad_mask_98.jpg,BAD_MASK,223,70,246,222,280,142,355,155


In [2]:
# first of all try cropping just 1 image, then you can iterate through them and get all the things you want!

try:
    for i in range(700):
        example = dataframe['image'][i]
        example_image = cv2.imread(example)
        face_x = dataframe['bbox_x'][i]
        face_y = dataframe['bbox_y'][i]
        face_w = dataframe['bbox_w'][i]
        face_h = dataframe['bbox_h'][i]
        folder = dataframe['class'][i]
        index = dataframe.index[i]
        face_only = example_image[face_y:face_y+face_w, face_x:face_x+face_h]
        face_resized = cv2.resize(face_only, (224,224), interpolation=cv2.INTER_AREA)
        cv2.imwrite(f'faces/{folder}/image_{index}.jpg', face_resized)
except:
    pass
# face extractor is done and ready to save and use
# you can specify a path and then save your faces to certain folders, let's start working on model

In [16]:
# haar cascade to extract faces from video
class FaceDetector:
    def __init__(self, bottom, top):
        self.bottom = bottom
        self.top = top
    def detect(self):
        frontalface_default = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
        frontalface_alt = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
        frontalface_alt2 = cv2.CascadeClassifier('haarcascade_frontalface_alt2.xml')
        profileface = cv2.CascadeClassifier('haarcascade_profileface.xml')
        cap = cv2.VideoCapture(0)
        f = 0
        while(cap.isOpened()):
            ret, frame = cap.read()
            if ret == True and f%10==0:
                gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                frontalfaces_default = frontalface_default.detectMultiScale(gray_frame, 1.03, 1)
                frontalfaces_alt2 = frontalface_alt2.detectMultiScale(gray_frame,1.03,1)
                profilefaces = profileface.detectMultiScale(gray_frame,1.03,1)
                for (x,y,w,h) in frontalfaces_default:
                    if w & h > self.bottom and w & h < self.top:
                        frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
                        face = frame[y:y+h,x:x+w]
                        face_resized = cv2.resize(face_only, (224,224), interpolation=cv2.INTER_AREA)
                        # predict here on face_resized
                for (x,y,w,h) in frontalfaces_alt2:
                    if w & h > self.bottom and w & h < self.top:
                        frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
                        face = frame[y:y+h,x:x+w]
                        face_resized = cv2.resize(face_only, (224,224), interpolation=cv2.INTER_AREA)
                        # predict here on face_resized
                for (x,y,w,h) in profilefaces:
                    if w & h > self.bottom and w & h < self.top:
                        frame = cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0))
                        face = frame[y:y+h,x:x+w]
                        face_resized = cv2.resize(face_only, (224,224), interpolation=cv2.INTER_AREA)
                        # predict here on face_resized
                cv2.imshow('face', frame)
                if cv2.waitKey(25) & 0xFF == ord('q'):
                    break
            f+=1
        cap.release()
        cv2.destroyAllWindows()


In [18]:
detector = FaceDetector(100, 200)
detector.detect()